In [125]:
import pandas as pd
from sklearn.model_selection import train_test_split
from optbinning import ContinuousOptimalBinning

from tensorflow import keras
from tensorflow.keras import layers
import shutil
# from tensorflow_docs.vis import embed
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import imageio
# import img_tiling # там склеиваются картинки из 4 в 1 квадрат

In [42]:
def get_picture(path, index):
    img = imageio.imread(path+'/'+str(index) + '.png')
    return img

In [2]:
df = pd.read_csv('houses.csv')

In [36]:
df['id'] = df.index

In [80]:
df['target1'] = pd.qcut(df['price'], q=20)

In [ ]:
# df.to_csv('houses_img.csv', index=False)

In [ ]:
df['image'] = df['id'].apply(lambda x: get_picture('img', x+1))

In [48]:
df['img_path'] = df['id'].apply(lambda x: 'img/'+str(x) + '.png')

In [63]:
# split df to train test
X_train, X_valid, y_train, y_valid = train_test_split(df.drop('price', axis=1), df['price'], test_size=0.2, random_state=42)

In [64]:
y_train.head()

284     589900
347     106000
440     639000
421    5858000
149    1795000
Name: price, dtype: int64

Выбросы уходят

In [65]:
y_train = y_train.clip(lower=y_train.quantile(0.05), upper=y_train.quantile(0.95))

In [66]:
variable = "square"
y = X_train[variable].values
x = y_train

In [67]:
optb = ContinuousOptimalBinning(name=variable, dtype="numerical")

In [68]:
optb.fit(x, y)

ContinuousOptimalBinning(name='square')

In [69]:
optb.status

'OPTIMAL'

In [70]:
optb.splits

array([  82600.,  135950.,  182450.,  292400.,  454000.,  541500.,
        690000.,  749450., 1191500.])

In [71]:
binning_table = optb.binning_table
binning_table.build()

,Bin,Count,Count (%),Sum,Std,Mean,Min,Max,Zeros count,WoE,IV
0,"(-inf, 82600.00)",22,0.051402,26356.0,230.360192,1198.000000,960.0,1680.0,0,-1163.922897,59.827812
1,"[82600.00, 135950.00)",40,0.093458,53727.0,190.456936,1343.175000,900.0,1766.0,0,-1018.747897,95.210084
2,"[135950.00, 182450.00)",22,0.051402,31590.0,274.163506,1435.909091,854.0,1930.0,0,-926.013806,47.598841
3,"[182450.00, 292400.00)",35,0.081776,64212.0,644.586117,1834.628571,1005.0,3768.0,0,-527.294326,43.119863
4,"[292400.00, 454000.00)",55,0.128505,105080.0,1222.369916,1910.545455,701.0,9583.0,0,-451.377443,58.004111
5,"[454000.00, 541500.00)",57,0.133178,145239.0,1220.14922,2548.052632,951.0,9536.0,0,186.129734,24.788306
6,"[541500.00, 690000.00)",70,0.163551,185615.0,1091.105233,2651.642857,766.0,7627.0,0,289.719960,47.384106
7,"[690000.00, 749450.00)",24,0.056075,65697.0,967.379183,2737.375000,1056.0,4406.0,0,375.452103,21.053389
8,"[749450.00, 1191500.00)",61,0.142523,170834.0,1080.355763,2800.557377,1084.0,6000.0,0,438.634480,62.515662
9,"[1191500.00, inf)",42,0.098131,162553.0,1193.750083,3870.309524,1560.0,7000.0,0,1508.386627,148.019248


In [72]:
x_transform_bins = optb.transform(x, metric="bins")

In [73]:
X_train['target'] = x_transform_bins

In [74]:
X_train.head()

,rooms,baths,square,post,id,image,img_path,target
284,5,5.0,3816,92880,284,"[[[225, 223, 210], [214, 212, 199], [206, 204,...",img/284.png,"[541500.00, 690000.00)"
347,2,2.0,1440,92276,347,"[[[93, 67, 44], [92, 66, 43], [92, 66, 43], [9...",img/347.png,"[82600.00, 135950.00)"
440,4,3.0,1625,93510,440,"[[[142, 100, 62], [142, 100, 62], [141, 101, 6...",img/440.png,"[541500.00, 690000.00)"
421,4,3.0,2454,93510,421,"[[[226, 223, 204], [215, 212, 193], [208, 205,...",img/421.png,"[1191500.00, inf)"
149,4,4.5,4038,92677,149,"[[[151, 146, 140], [152, 147, 141], [153, 149,...",img/149.png,"[1191500.00, inf)"


In [15]:
y_train.head()

284     589900
347     106000
440     639000
421    5858000
149    1795000
Name: price, dtype: int64

Нам мало 9 бинов, там разница в процентах ошибки очень большая

In [75]:
X_train['target1'] = pd.qcut(y_train, q=20)

In [77]:
X_train['target1'].value_counts()

(449980.0, 499000.0]         24
(81299.999, 82210.0]         22
(914750.0, 1161400.0]        22
(679900.0, 731250.0]         22
(575600.0, 625000.0]         22
(548835.0, 575600.0]         22
(1499422.078, 1499649.35]    22
(243750.0, 330500.0]         22
(104480.0, 139520.0]         22
(408305.0, 449980.0]         21
(330500.0, 408305.0]         21
(499000.0, 529000.0]         21
(82210.0, 104480.0]          21
(187500.0, 243750.0]         21
(139520.0, 187500.0]         21
(731250.0, 839000.0]         21
(839000.0, 914750.0]         21
(1161400.0, 1499422.078]     21
(625000.0, 679900.0]         20
(529000.0, 548835.0]         19
Name: target1, dtype: int64

Другое дело

In [ ]:
# df2 = X_train
# df2['price'] = y_train
# df2 = df2.drop('target', axis=1)

In [87]:
# df2['target1'] = df2['target1'].astype('str')
# df2.to_csv('house_classes.csv', index=False)


In [89]:
df.head(1)

,rooms,baths,square,post,price,id,image,img_path,target1
0,4,4.0,4053,85255,869500,0,"[[[159, 122, 96], [166, 132, 105], [166, 139, ...",img/0.png,"(824999.2, 898800.0]"


In [100]:
# get value of first row column 'target1'
df['target1'].iloc[0]

Interval(824999.2, 898800.0, closed='right')

In [148]:
df[['target', 'price']].iloc[10]

target    (200000.0, 300000.0]
price                   519200
Name: 10, dtype: object

In [157]:
def check_interval(value, intervals):
    ans = 0
    for interval in intervals:
        if interval.left <= value <= interval.right:
            ans =  interval            
    if ans != 0:
        return ans
    if value < 100000:
        return pd.Interval(0, 100000)
    else:
        return pd.Interval(200000, 300000)    


In [102]:
intervals = X_train['target1'].unique()

In [158]:
df['target2']  = df['price'].apply(lambda x: check_interval(x, intervals))

In [159]:
# df.head(3)
df['target'] = df['target2']

In [161]:
# df = df.drop(['target2'], axis=1)
df.to_csv('house_classes.csv')

In [162]:
# create folder for every target value
df['target'] = df['target'].astype(str)
for target in df['target'].unique():
    try:
        os.makedirs(f'img/{target}', exist_ok=True)
    except:
        print('exist')

In [128]:
df['img_path'] = df['id'].apply(lambda x: 'img/'+str(x+1)+'.png')

In [163]:
# if row equal target value, copy image to folder
for index, row in df.iterrows():
    shutil.copy(row['img_path'], f'img/{row["target"]}')

In [131]:
df.head(10)

,rooms,baths,square,post,price,id,image,img_path,target
0,4,4.0,4053,85255,869500,0,"[[[159, 122, 96], [166, 132, 105], [166, 139, ...",img/1.png,"(839000.0, 914750.0]"
1,4,3.0,3343,36372,865200,1,"[[[227, 203, 175], [226, 202, 174], [226, 202,...",img/2.png,"(839000.0, 914750.0]"
2,3,4.0,3923,85266,889000,2,"[[[145, 97, 49], [133, 84, 41], [107, 59, 21],...",img/3.png,"(839000.0, 914750.0]"
3,5,5.0,4022,85262,910000,3,"[[[245, 202, 160], [245, 202, 160], [249, 206,...",img/4.png,"(839000.0, 914750.0]"
4,3,4.0,4116,85266,971226,4,"[[[217, 235, 247], [231, 249, 255], [237, 255,...",img/5.png,"(914750.0, 1161400.0]"
5,4,5.0,4581,85266,1249000,5,"[[[183, 151, 130], [176, 145, 125], [166, 138,...",img/6.png,"(1161400.0, 1499422.078]"
6,3,4.0,2544,85262,799000,6,"[[[211, 176, 84], [231, 196, 104], [220, 185, ...",img/7.png,"(731250.0, 839000.0]"
7,4,5.0,5524,85266,1698000,7,"[[[252, 252, 244], [253, 253, 245], [254, 254,...",img/8.png,NaN
8,3,4.0,4229,85255,1749000,8,"[[[207, 205, 206], [208, 206, 207], [209, 209,...",img/9.png,NaN
9,4,5.0,3550,85262,1500000,9,"[[[219, 205, 176], [216, 202, 173], [211, 197,...",img/10.png,NaN
